In [ ]:
import warnings

import pandas as pd
import pytest
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

warnings.filterwarnings("ignore")

df = pd.read_csv("https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/diabetes.csv")
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector

import numpy as np

from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import Pipeline

In [ ]:
column_trans = ColumnTransformer(
    [
     ('minmaxscaler', MinMaxScaler(feature_range=(0, 0.5)), make_column_selector(dtype_include=np.number)),
    ],
    remainder = 'drop',
)

X_trans = column_trans.fit_transform(df)

names_col = list(df.columns)

data = pd.DataFrame(X_trans, columns = names_col)
data.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.333333,0.5,0.291322,0.274648,0.147059,0.128486,0.103896,0.141044,0.281109,0.219697,0.196262
1,0.241667,0.0,0.074380,0.176056,0.210784,0.153386,0.311688,0.070522,0.111222,0.083333,0.077882
2,0.441667,0.5,0.258264,0.218310,0.144608,0.129482,0.123377,0.141044,0.248292,0.204545,0.180685
3,0.041667,0.0,0.150826,0.154930,0.247549,0.223606,0.116883,0.211566,0.286468,0.234848,0.281931
4,0.258333,0.0,0.103306,0.274648,0.232843,0.208665,0.194805,0.141044,0.181185,0.166667,0.171340


In [ ]:
y = data.pop('target')
X = data.copy()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_true_train, y_true_test = train_test_split(
    X,
    y,
    test_size=0.30,
    random_state=170,
)

X_test, X_val, y_true_test, y_true_val = train_test_split(
    X,
    y,
    test_size=0.50,
    random_state=170,
)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
estimators = [("m", MLPRegressor(max_iter=100, learning_rate='adaptive',learning_rate_init=0.001,
                                 activation='identity', solver='adam'))]

pipeline = Pipeline(
    steps=estimators,
    verbose=False
)

In [ ]:
param_grid = [
              {
                  'm__hidden_layer_sizes': [(1,),(2,),(3,),(4,),(5,)],
                  'm__random_state':[1000,1001,1002,1003,1004,1005],
                  'm__activation': ['identity', 'logistic','relu'],   #,'tanh'
                  'm__solver': ['sgd', 'adam'],  # , 'lbfgs'
               }          
]

grid = GridSearchCV(
    estimator = pipeline,
    param_grid = param_grid,
    cv = 5
    )

grid.fit(X_train, y_true_train)

print(grid.best_params_)

{'m__activation': 'identity', 'm__hidden_layer_sizes': (3,), 'm__random_state': 1004, 'm__solver': 'adam'}


In [ ]:
y_pred_test = grid.predict(X_test)

mse_val = mean_squared_error(y_true_test, y_pred_test)
mse_val

0.009986571430374515

In [ ]:
y_pred_val = grid.predict(X_val)

mse_val = mean_squared_error(y_pred_val, y_true_val)
mse_val

0.007901536863209734

In [ ]:
#
# Use la muestra (X_train, y_true_train) para la estimación
# de los pesos óptimos de la red neuronal.
#
# Seleccione el modelo óptimo como aquel que minimice el error
# cuadrático medio para la muestra (X_test, y_true_test).
#
# Considere únicamente modelos desde una (1) hasta (5) 
# neuronas en la capa oculta. Considere únicamente las
# siguientes semillas para inicializar la red neuronal
# 1000, 1001, 1002, 1003, 1004, 1005.
#
# Compute el error cuadrático medio para la muestra
# (X_val, y_true_val). Esta muestra representa la operación
# del modelo en productio
# 
# Rta/
# True
#

# >>> Inserte su codigo aquí >>>

# <<<

# ---->>> Evaluación ---->>>
pytest.approx(mse_val, 0.0001) == 0.009535

False